In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import json
import datetime

plt.rcParams["figure.figsize"] = (20, 5)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
data = np.genfromtxt("data/SEG_2_train_set.csv", delimiter="\n", dtype=np.int64)
data

array([        -1,         -1,         -1, ..., 5992956672, -753929088,
        639534672], dtype=int64)

In [26]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        ## No Reshape in this version (as Word Embedding layer does that)
        # Reshape data from (history_size,) to (history_size, n_feature)
        data.append(dataset[indices])
        labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [27]:
x_train, y_train = generate_timeseries(data, 0, None, 16, 8)

In [31]:
x_train[0].shape

(16,)

In [16]:
vocab_size = len(pd.Series(data).value_counts().index)
vocab_size

17387

In [32]:
embdding_dim = 256

In [33]:
model = keras.models.Sequential()
model.add(keras.layers.Embedding(vocab_size, embdding_dim, input_shape=(None,)))

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(5)))  #x_train.shape[-2:] , input_shape=[16, 5] , return_sequences=True
model.add(tf.keras.layers.RepeatVector(8))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(5, return_sequences=True)))
model.add(keras.layers.TimeDistributed(tf.keras.layers.Dense(5, activation="softmax")))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
model.predict(x_train[0])

InvalidArgumentError:  indices[0,0] = -1 is not in [0, 17387)
	 [[node sequential/embedding/embedding_lookup (defined at <ipython-input-34-6551796d9b48>:1) ]] [Op:__inference_predict_function_89]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/embedding/embedding_lookup:
 sequential/embedding/embedding_lookup/83 (defined at C:\ProgramData\Anaconda3\lib\contextlib.py:112)

Function call stack:
predict_function
